In [1]:
#IV Load the data for a station and organize for precipitation model
import os as os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import csv
from functools import reduce

In [2]:
#Paths
#Need to set up based on who is running
usr = "JH"
if usr == "PK":
    ipd01 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\2024\\"
    ipd02 = "D:\\CodeLibrary\\Python\\weathermetrics\\data\\weathermetrics\\"
    ipd03 = ""
    ipd04 = ""
    
if usr == "JH":
    ipd01 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\2024\\"
    ipd02 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\"
    ipd03 = "\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\Py_S4_v02_JHH\\NCEI_data\\"
    ipd04 = "\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\Py_S4_v02_JHH\\NCEI_parquet_files\\"


In [3]:
def fn_select_data_vectors_pr07(stid_keys,nstations,nhours,nmonths_row,nmonths_col):
    #Get the matrix of data to input into marix
    #The S4 model
    # 1. Read in combined data files for multiple stations
    # 2. Find the common dates that will be used to align the matrices
    #    Keep only the values that fit the common starting date 

    # 1. Read in data files and get start date v_date... for each
    # Check that data exist for all metrics

    df_stid = stid_keys
    df_stid2 = df_stid.iloc[:, [1]]
    df_stid2 = df_stid2.drop_duplicates() # unique()
    
    n_rows_stid = min(len(df_stid2),nstations)
    
    #Matrix in which measures are stored
    n_vmt_rows = nhours*30*max(nmonths_row,nmonths_col)
    
    data_list = []
    date_max_list =[]
    
    #1. Read in data files
    for i in range(0, n_rows_stid):
    #for i in range(0, 28):
        infile = ipd03 + str(df_stid2.iloc[i,0]) + "_model_data_combined.csv"
        print(infile)
        
        #Eliminate fully blank lines
        cleaned_lines = []
        with open(infile, 'r', newline='') as infile:
            reader = csv.reader(infile)
            for row in reader:
                # Example: Skip rows that are entirely empty or have only blank 'name'
                if any(cell.strip() for cell in row) and (len(row) < 1 or row[0].strip() != ''): # Assuming 'name' is the first column
                    cleaned_lines.append(row)
        
        df_metrics = pd.DataFrame(cleaned_lines[1:], columns=cleaned_lines[0]) # Assuming first row is header

        df_metrics['str_hr'] = df_metrics['nHOUR']  #.apply(lambda x: f'{x:02d}')
        df_metrics['date_hr'] = pd.to_datetime(df_metrics['Date_YYYYMMDD'] + ' ' + df_metrics['str_hr'])
        df_metrics['month_number'] = df_metrics['date_hr'].dt.month
        df_metrics['week_number'] = df_metrics['date_hr'].dt.isocalendar().week
        df_metrics['stid'] = str(df_stid2.iloc[i,0])
        data_list.append(df_metrics)

    #2. Align dates across data files
    for i in range(0, n_rows_stid):
        df = data_list[i]
        max_date = max(df['date_hr'])     
        date_max_list.append(max_date)
    
    date_filter = min(date_max_list)

    for i in range(0, n_rows_stid):
        df = data_list[i]

        filtered_df = df[df['date_hr'] < date_filter]
        filtered_df = filtered_df.iloc[(len(filtered_df) - n_vmt_rows):len(filtered_df)]
        
        idx_col = filtered_df.columns
        idx_col = str(df_stid2.iloc[i,0]) + '_' + idx_col
        filtered_df.columns = idx_col

        print(filtered_df.shape)
        
        data_list[i] = filtered_df
        
    return data_list[0:n_rows_stid]

In [4]:
def fn_select_data_vectors_pr07xxx(stid_keys,nstations,nhours,nmonths_row,nmonths_col):
    #Get the matrix of data to input into marix
    #The S4 model
    # 1. Read in combined data files for multiple stations
    # 2. Find the common dates that will be used to align the matrices
    #    Keep only the values that fit the common starting date 

    # 1. Read in data files and get start date v_date... for each
    # Check that data exist for all metrics

    df_stid = stid_keys
    df_stid2 = df_stid.iloc[:, [0, 3, 4]]  #df_stid[df_stid['target_station'] == stid_target]
    df_stid2 = df_stid2.drop_duplicates() # unique()

    n_rows_stid = min(len(df_stid2),nstations)
    
    #Matrix in which measures are stored
    n_vmt_rows = nhours*30*max(nmonths_row,nmonths_col)
    #vmt_full_set = np.empty(n_vmt_rows,np.float16)
    #is_first = 1
    
    data_list = []
    date_max_list =[]

    #1. Read in data files
    for i in range(0, n_rows_stid):
        infile = ipd03 + str(df_stid2.iloc[i,0]) + "_model_data_combined.csv"
        print(infile)
        df_metrics = pd.read_csv(infile)
        df_metrics['str_hr'] = df_metrics['nHOUR'].apply(lambda x: f'{x:02d}')
        df_metrics['date_hr'] = pd.to_datetime(df_metrics['Date_YYYYMMDD'] + ' ' + df_metrics['str_hr'])
        df_metrics['month_number'] = df_metrics['date_hr'].dt.month
        df_metrics['week_number'] = df_metrics['date_hr'].dt.isocalendar().week
        df_metrics['stid'] = str(df_stid2.iloc[i,0])
        data_list.append(df_metrics)

    #2. Align dates across data files
    for i in range(0, n_rows_stid):
        df = data_list[i]
        max_date = max(df['date_hr'])     
        date_max_list.append(max_date)
    
    date_filter = min(date_max_list)

    for i in range(0, n_rows_stid):
        df = data_list[i]
        #place holder for filtering to a specific end date
        #date_filter = pd.to_datetime("2023-8-11" + ' ' + "17")
        filtered_df = df[df['date_hr'] < date_filter]
        filtered_df = filtered_df.iloc[(len(filtered_df) - n_vmt_rows):len(filtered_df)]
        
        idx_col = filtered_df.columns
        idx_col = str(df_stid2.iloc[i,0]) + '_' + idx_col
        filtered_df.columns = idx_col

        print(filtered_df.shape)
        
        data_list[i] = filtered_df
        
    return data_list[0:n_rows_stid]

In [4]:
def fn_create_data_matrix_pr07(lst_data,tgt_metric,nstations,nmetrics,nmonths_col,ndays_col):
    #Create a matrix for precipitation data that feeds
    #The S4 model
    # 1. Get data list and select target metric
    #Two choices now
    #    - prec
    #    - temp
    # 2.  Create output and move data into approprate cells

    # 1. get data list details and reduce to metrics
    data_vectors_list = []
    dt_rows = len(lst_data[0])
    df00 = []

    dt_rows_minus_1 = dt_rows - 1
    
    if tgt_metric == "prec": tgt_cols = [10,2,3,4,5,6,8,9,11,12]
    if tgt_metric == "temp": tgt_cols = [10,3,2,4,5,6,8,9,11,12]
    non_metrics = 4  #These are the non sensor data brought in to analysis (e.g., day of year, month, ...)
    
    #Reset the indices as all records are ordered by date and hour
    #But stations have missing records so not everything aligns
    #Need to use the target station (station 0) as the standard and left join th other fields
    for i in range(nstations):
        tdt = lst_data[i].iloc[:,tgt_cols]
        tdt = tdt.iloc[::-1]
        tdt.reset_index(drop=True, inplace=True)
        first_column_name = tdt.columns[0]
        tdt.rename(columns={first_column_name: 'key_date_hr'}, inplace=True)
        df00.append(tdt)
        fn = f"1. Done with setting up station -  {i}"
        print(fn)
        #print(tdt)
        
    
    #Take the list of station data, df00, and paste side by side
    #to produce rs which is a large matrix of data for all stations 
    join_key = 'key_date_hr'
    rs = reduce(lambda left, right: pd.merge(left, right, on=join_key, how='left'), df00)
    
    #rs = pd.concat(df00, axis=1)  #old way that did merge based on record index
    #rs has each row being a dayXhour for all stations for the stations full metrics
    #full metrics is len(tgt_cols) - 1
    fn = f"2. Shape of combined data including all stations"
    print(fn)
    print(rs.shape)
    
    
    #2. Create output matrix and move data into appropriate cells
    
    nmetrics_and_non_metrics = nmetrics + non_metrics
    fn = f"nmetrics_and_non_metrics {nmetrics_and_non_metrics}"
    print(fn)
    ndataelements = nstations*(nmetrics_and_non_metrics)
    fn = f"ndataelements - stations by metrics {ndataelements}"
    print(fn)
    mt_cols = 1 + ndataelements*ndays_col*24
    fn = f"mt_cols 1 + #data elements*ndays_col*24 {mt_cols}"
    print(fn)
    mt_rows = len(rs)
    
    #mt_rows = 1000  #keep here for development until done
    mt_full_set = np.empty((mt_rows,mt_cols ),np.float16)
    
    #tgt_cols = range(1,181)
    
    #Create matrix of data
    #tgt_cols_num = len(tgt_cols)
    for i in range(0,mt_rows):
        mt_full_set[i,range(0,ndataelements)] = rs.iloc[i,range(1,ndataelements+1)].apply(pd.to_numeric, errors='coerce')
 
    fn = f"3. Full mt setup with initial data for the dayXhour only"
    print(fn)  
    print(mt_full_set.shape)
    
    #Add additional column sets
    lpi = mt_rows - ndays_col*24 - 1  #number of rows in matrix - the data elements in a column... 
                                    #ensures all records have data
    #limit of rows in data set so all matrix rows have data
    lpj = ndays_col*24 -1   #number of colum sets in columns
    
    #The row I am operating on
    for i in range(0,lpi):
        #The column I am going to fill
        for j in range(0,lpj):
        
            row_get = i + j + 1              #For every new column in current row we have to go down one more row
            col_get_start = 0                #data retrieved starting column 1
            col_get_end = ndataelements - 1
            
            row_put = i
            col_put_start = ndataelements*(row_get - row_put)
            col_put_end = col_put_start + ndataelements - 1
            
            mt_full_set[row_put,col_put_start:col_put_end] = mt_full_set[row_get,col_get_start:col_get_end]
     
    fn = f"4. mt_full_set with ndays of data added to the columns"
    print(fn)  
    print(mt_full_set.shape)
           
    return mt_full_set #rs #mt_full_set

In [7]:
def fn_create_data_matrix_pr07xxx(lst_data,tgt_metric,nstations,nmetrics,nmonths_col):
    #Create a matrix for precipitation data that feeds
    #The S4 model
    # 1. Get data list and select target metric
    #Two choices now
    #    - prec
    #    - temp
    # 2.  Create output and move data into approprate cells

    # 1. get data list details and reduce to metrics
    data_vectors_list = []
    dt_rows = len(lst_data[0])
    df00 = []

    dt_rows_minus_1 = dt_rows - 1
    
    if tgt_metric == "prec": tgt_cols = [8,2,3,4,5,6,9,10]
    if tgt_metric == "temp": tgt_cols = [8,3,2,4,5,6,9,10]
    
    #Reset the indices as all records are ordered by date and hour
    for i in range(nstations):
        df00.append(lst_data[i].iloc[:,tgt_cols])
    
    df01 = [df.reset_index(drop=True) for df in df00]
    rs = pd.concat(df01, axis=1)
    print(rs.shape)
    #print(rs.iloc[1:10,:])
    #Order from most recent to most distant in time
    rs = rs.iloc[::-1]
    #print(rs.iloc[1:10,:])
    
    #2. Create output matrix and move data into appropratie cells
    
    nmetrics_mon_week = nmetrics + 2
    mt_cols = 1 + nstations*(nmetrics_mon_week)*nmonths_col*30
    mt_rows = len(rs) #- nmonths_col*30*24 - 1
    
    mt_full_set = np.empty((mt_rows,mt_cols ),np.float16)
    
    #rng = mt_rows
    #Set up first column set of data
    #tgt_cols = [1,2,3,4,5,7,8,9,10,11,13,14,15,16,17,19,20,21,22,23]
    tgt_cols = [1,2,3,4,5,6,7,9,10,11,12,13,14,15,17,18,19,20,21,22,23,25,26,27,28,29,30,31]
    
    
    tgt_cols_num = len(tgt_cols)
    for i in range(0,mt_rows):
        mt_full_set[i,range(0,tgt_cols_num)] = rs.iloc[i,tgt_cols]
    

    #Add additional column sets
    nc = nstations*nmetrics_mon_week
    lpi = mt_rows - nmonths_col*30
    #limit of rows in data set so all marix rows have data
    lpj = nmonths_col*30 - nc  #number of colum sets in columns
    
    for j in range(0,lpj):
        for i in range(0,lpi):
            row_get = i + j + 1
            col_get_start = 0
            col_get_end = nc

            row_put = i
            col_put_start = nc*(j+1)
            col_put_end = col_put_start + nc

            mt_full_set[row_put,col_put_start:col_put_end] = mt_full_set[row_get,col_get_start:col_get_end]
    
    return mt_full_set

In [5]:
#functions to identify stations in network that have rain sensor and rain
def cnt_ge0(row):
        cnt_gt_0 = sum(row >= 0)
        return cnt_gt_0

def cnt_gt0(row):
        cnt_gt_0 = sum(row > 0)
        return cnt_gt_0

In [34]:
def fn_prec_ex01(mtx01,nstations,nmetrics,dayscol):
    
    df = pd.DataFrame(mtx01)
    ncols_orig = df.shape[1]
    print(f"Number of mxta columns {ncols_orig}")
    
    
    #Does target have a finite value
    df['tgt_isfinite'] = np.isfinite(df.iloc[:,0])
    
    #moving average of target value
    window_hrs = 6
    nm1 = 'tgt_roll_sum_per' + str(window_hrs)
    df[nm1] = df.iloc[:,0].rolling(window=window_hrs, min_periods=1).sum()
    nm2 = 'tgt_roll_round_int' + str(window_hrs)
    df[nm2] = round(df[nm1],0)
    
    #nstations = 20
    #dayscol = 7
    #nmetrics = 9
    curr_per = 1   #current period for which average is to be taken, must be 1 or greater
    tot_per = (dayscol*24 - 1)  #days by 24 hours minus 1
    

    for iper in range(1,tot_per,1):
        vc_start = iper*nstations*nmetrics
        vc_end = vc_start+ nstations*nmetrics - 1
        shift_right = 0
        vc_num = np.arange(vc_start,vc_end,nmetrics) + shift_right

        nm = 'avg'+ str(iper)
        df[nm] = df.iloc[:,vc_num].mean(axis=1,skipna=True)

        nm = 'cnt_ge_zero'+ str(iper)
        jjge0 = df.iloc[:,vc_num].apply(cnt_ge0,axis=1)
        df[nm] = jjge0

        nm = 'cnt_gt_zero'+ str(iper)
        jjgt0 = df.iloc[:,vc_num].apply(cnt_gt0,axis=1)
        df[nm] = jjgt0
    
        ncols_final = df.shape[1]
        new_cols = range(ncols_orig+1,ncols_final,1)
        old_cols = range(180,ncols_orig,1)
        vc_keep = [0]+list(new_cols) + list(old_cols)
    
    rtn_df = df  #df.iloc[:,vc_keep]
    rtn_df = rtn_df.iloc[:,vc_keep]
        
    return rtn_df

In [29]:
def fn_create_tgt_matrix_pr07(mtx_data,tgt_metric,tgt_mod,nstations,nmetrics,ndays_col):
    
    mtx01 = mtx_data
    nrows = mtx01.shape[0]
    ncols = mtx01.shape[1]
        
    if tgt_metric == "prec" and tgt_mod == "prec_ex01": 
        rtn_matrix = fn_prec_ex01(mtx01,nstations,nmetrics,ndays_col)
    
    #nrows = rtn_matrix.shape[0]
    #ncols = rtn_matrix.shape[1]
    
    #1. Modify tgt variable
    # Shift the features down 1 row so they always occur before the tgt
    #vc_tgt = mtx01[0:(nrows-1),0].copy()
    #mt_feat = mtx01[1:(nrows),0:(ncols)].copy()
    #mt_feat[:,0]=vc_tgt
    
    #Modify the target variable depending on the metric (precipitation or temperature)
    #Modify tgt depending on tgt_mod, options could be running totals, moving averages, 
        
    #2. Check that all columns and rows have some type of data
    
    
        
    #return the matrix for analysis
        
    return rtn_matrix

In [8]:
def fn_get_nearby_stations(stid_target, target_file,in_wd):

    dt01 = target_file[target_file['tgt_stid']==stid_target]
    dt01 = dt01[['feat_stid','dist_km']]
    
    vc_stid = dt01['feat_stid'].to_numpy()
    vc_stid = np.insert(vc_stid,0,tgt_stid)
    #vc_stid = [stid_target] + [vc_stid]
    
    #Need to create more combined data sets
    lst_files = os.listdir(in_wd)
    df_files = pd.DataFrame(lst_files,columns=["files"])
    df_files['stid'] = df_files['files'].str[:11]
    df_files['istarget'] = df_files['stid'].str.contains(stid_target,case=False)
    df_files['file_type'] = df_files['files'].str[23:31]
    df_files['isintarget'] = df_files['stid'].isin(vc_stid)
    
    df_files = df_files[df_files['file_type']=="combined"]
    df_files = df_files[df_files['isintarget']==True]
    
    mrg_df = pd.merge(df_files, dt01,how='left',left_on='stid',right_on='feat_stid')
    mrg_df = mrg_df.drop(columns=['feat_stid','isintarget','file_type'])
    mrg_df = mrg_df.sort_values(by='dist_km',na_position='first')
    mrg_df = mrg_df.reset_index(drop=True)
    mrg_df.columns = ['fnames','st_id','is_target_station','dist_km']
    
    return mrg_df

In [ ]:
#STEP 1.  Select a station using ID to get its related stations


#Load df of target stations and those close by
#74486094789 is JFK airport
#74486454787 is Farmingdale
#72505004781 is Macarthur
#72502014734 is Newark
#Select the target station for which predictions will be made
#Select the target station for which predictions will be made
dt_trg_set2 = ipd02 + "Station_Pairs_LE_100km_Info.csv"
dt_analysis_set2 = pd.read_csv(dt_trg_set2)
#Governs the target station in subsequent modules
tgt_stid =  "74486094789"

#Nearby stations within 100km
dt_analysis_set_keys = fn_get_nearby_stations(tgt_stid,dt_analysis_set2,ipd03)

nstations_in_100km = len(dt_analysis_set_keys) #number of stations within 100 km
num_st_keep = 20   #num of stations to keep for analysis
nstations = min(nstations_in_100km,num_st_keep)

#analysis_set = dt_trg_set
#dt_analysis_set = pd.read_csv(analysis_set)
#dt_analysis_set['path'] = ipd02
#dt01 = dt_analysis_set[dt_analysis_set['target_station']==int(st_id)]
#uniq_stat = dt01['vect_close_bkt2'].unique()[:2]
#uniq_station = dt01['st_id'].unique()[:num_st_keep]
#dt02 = dt01[dt01['st_id'].isin(uniq_station)]

dt_analysis_set_keys


In [10]:
#STEP 2.  Use elements created in STEP 1 to get a data matrix for analysis
#All the available stations which have all three metrics needed for model
#vc_stid = dt_analysis_set_keys[['st_id']].drop_duplicates()

#Organize the data based on the nature of the data so they all conform to the same size matrices
#The metrics included are
#  AT - air temperature in Celcius
#  RH - relative humidity in Percent
#  PR - precipitation in MM

#Put them all in appropriate matrices and create list of metrics
#Data files are of two types
#    -  24 hour * 365 days * n years  (AT, PR)
#    -  365 days * n years (RH)

#Parameters governing data matrix
stid_target = tgt_stid
stid_keys = dt_analysis_set_keys

ipd = ipd02
nhours = 24
ndays = 365
nmonths_row = 240
nmonths_col = 2 #governs the number of months predicting that are needed
ndays_col = 7  #governs the number of days predicting that are needed
nmetrics = 5
tgt_metric =  "prec" #"prec"  "temp"
tgt_mod = "prec_ex01"


In [ ]:

#1. Get the relevant data in a list
lst_data = fn_select_data_vectors_pr07(stid_keys,nstations,nhours,nmonths_row,nmonths_col)
fn = f"lst data done"
print(fn)


In [ ]:

#2. Construct a matrix from most recent to most distant from the list data
mtx_data = fn_create_data_matrix_pr07(lst_data,tgt_metric,nstations,nmetrics,nmonths_col,ndays_col)
fn = f"mtx data done"
print(fn)

#Save the mtx_data to a folder
#This lets us start at Step 3 for iterative runs
print(ipd04)
rtn_mtx_pqt = pd.DataFrame(mtx_data)
print(rtn_mtx_pqt.shape)
fn = f"{ipd04}{tgt_metric}_mtx_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
print(fn)
rtn_mtx_pqt.to_parquet(fn, index=False)



In [ ]:
#3.  Read in mtx_data file

#Get the mtx_data from a folder
#This lets us start at Step 3 for iterative runs
fn = '\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\Py_S4_v02_JHH\\NCEI_parquet_files\\prec_mtx_74486094789_stations_20_RowMn_240_ColDy_7.parquet'

# Read the Parquet file into a Pandas DataFrame
mtx_data = pd.read_parquet(fn)
print(mtx_data.shape)


(172800, 30241)


In [ ]:
#3.  Create a target variable and select feature variables

nmetrics = 9

s4_data = fn_create_tgt_matrix_pr07(mtx_data,"prec","prec_ex01",nstations,nmetrics,ndays_col)
print(f"Shape of s4_data {s4_data.shape}")
fn = f"s4 data done"
print(fn)


#Save the s4_data to a folder
#This lets us start at Step 3 for iterative runs
print(f"Done temp")
print(ipd04)
rtn_s4_data_pqt = pd.DataFrame(s4_data)
print(rtn_s4_data_pqt.shape)
fn = f"{ipd04}{tgt_metric}_s4data_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
print(fn)
rtn_s4_data_pqt.to_parquet(fn, index=False)



In [ ]:
jj = s4_data.iloc[0:1000,:]
jj


In [25]:
def transform_matrix(df, bins, labels):
    """
    Transforms the given matrix by binning the first column, dropping the first row and column,
    and adding the binned column as the first column.

    Parameters:
        matrix (pd.DataFrame): The input matrix.

    Returns:
        pd.DataFrame: The transformed matrix.
    """
    # Extract the first column (dependent variable)
    firstcolumn = df.iloc[:, 0].astype('float32')

    # Bin the first column
    binned_column = pd.cut(firstcolumn, bins=bins, labels=labels)

    # Drop the first first column
    df = df.iloc[:, 1:]

    # Add the binned column as the first column
    df.insert(0, 0, binned_column.iloc[0:].values)

    # Return the final matrix
    return df

# Example usage
# Example matrix (replace this with your actual matrix)
df = pd.DataFrame(s4_data).iloc[1:,] # Convert to DataFrame and drop the first row for binning
# Drop rows where the first column has missing values
# df = df.dropna(subset=[0])
fc = df.iloc[:,0].astype('float32') # Ensure the first column is float for binning
#DEL#bins = [-float('inf'), 10, 20, 25, float('inf')]

bins = [-float('inf'), 1, float('inf')]
# bins = [-float('inf'), 0] + list(pd.qcut(fc[fc > 0], q=4, retbins=True)[1][1:]) + [float('inf')]
labels = [0, 1]
# print(bins, labels)


# bins = [-float('inf'), 0] + list(pd.qcut(fc[fc > 0], q=4, retbins=True)[1][1:]) + [float('inf')]
#labels = [0, 1, 2, 3]
# print(bins, labels)

df_transformed = transform_matrix(df, bins, labels)
df_transformed = df_transformed[df_transformed[0]>= 0]
print("Dataset: ", df_transformed.shape)
df_transformed[0].value_counts(dropna=False).sort_index()

Dataset:  (172360, 5041)


0
0    166321
1      6039
Name: count, dtype: int64

In [26]:
def split_time_series_data(df, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    """
    Splits a time series dataset into training, validation, and test sets.

    Parameters:
        df (pd.DataFrame): The input time series dataset, ordered in descending dates.
        train_ratio (float): The proportion of data to use for training.
        val_ratio (float): The proportion of data to use for validation.
        test_ratio (float): The proportion of data to use for testing.

    Returns:
        tuple: A tuple containing three DataFrames (train, validation, test).
    """
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    # Calculate split indices
    n = len(df)
    test_end = int(n * test_ratio)    
    val_end = test_end + int(n * val_ratio)
    

    # Split the data
    test_data = df.iloc[:test_end].sort_index(ascending=False)
    val_data = df.iloc[test_end:val_end].sort_index(ascending=False)
    train_data = df.iloc[val_end:].sort_index(ascending=False)


    return train_data, val_data, test_data

# Example usage
train_data, val_data, test_data = split_time_series_data(df_transformed)
print("Training Set:", train_data.shape)
print(train_data[0].value_counts(dropna=False).sort_index())
print("Validation Set:", val_data.shape)
print(val_data[0].value_counts(dropna=False).sort_index())
print("Test Set:", test_data.shape)
print(test_data[0].value_counts(dropna=False).sort_index())

Training Set: (120652, 5041)
0
0    116260
1      4392
Name: count, dtype: int64
Validation Set: (17236, 5041)
0
0    16626
1      610
Name: count, dtype: int64
Test Set: (34472, 5041)
0
0    33435
1     1037
Name: count, dtype: int64


In [27]:
# Define the station ID and output directory
station_id = st_id
output_dir = ipd02


# Export datasets to CSV
train_data.to_parquet(f"{output_dir}{station_id}_train.parquet", index=False)
val_data.to_parquet(f"{output_dir}{station_id}_validation.parquet", index=False)
test_data.to_parquet(f"{output_dir}{station_id}_test.parquet", index=False)

# train_data.to_csv(f"{output_dir}{station_id}_train.csv", index=False)
# val_data.to_csv(f"{output_dir}{station_id}_validation.csv", index=False)
# test_data.to_csv(f"{output_dir}{station_id}_test.csv", index=False)

print("Datasets exported successfully.")

#python -m s4model --modelname 72406093721 --trainset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_train.parquet --valset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_validation.parquet --testset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_test.parquet --tabulardata --dependent_variable 0 --epochs 30

# python -m charts --df ../results/72508014740_Test_results_20250424_104121PM.csv --actual 0 --predicted Predicted 
# python -m charts --df ../results/72508014740_Test_results_20250427_100132PM.csv --actual 0 --predicted Predicted 
# python -m charts --df ../results/72508014740_Test_results_20250428_091259PM.csv --actual 0 --predicted Predicted 
# python -m charts --df ../results/72508014740_Test_results_20250428_094033PM.csv --actual 0 --predicted Predicted 
# python -m charts --df ../results/72508014740_Test_results_20250429_091538PM.csv --actual 0 --predicted Predicted

Datasets exported successfully.


In [ ]:
#Code to run at cmd
#python -m s4model --modelname 72406093721 --trainset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_train.parquet --valset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_validation.parquet --testset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_test.parquet --tabulardata --dependent_variable 0 --epochs 30


SyntaxError: invalid syntax (2106272906.py, line 1)

In [ ]:
# def find_first_non_nan(vector):
#     return next((i for i, x in enumerate(vector) if not math.isnan(x)), None)

# first_number_at1 = find_first_non_nan(vc_at1)
# first_number_rh1 = find_first_non_nan(vc_rh1)
# first_number_rh2 = find_first_non_nan(vc_rh2)
# first_number_rh3 = find_first_non_nan(vc_rh3)
# first_number_pr1 = find_first_non_nan(vc_pr1)

# #Number of missing data points
# print(f"AT1 Number of missing cells: {np.count_nonzero(np.isnan(vc_at1))}")
# print(f"AT1 Vector Length: {len(vc_at1)}")

# print(f"RH1 Number of missing cells: {np.count_nonzero(np.isnan(vc_rh1))}")
# print(f"RH1 Vector Length: {len(vc_rh1)}")

# print(f"PR1 Number of missing cells: {np.count_nonzero(np.isnan(vc_pr1))}")
# print(f"PR1 Vector Length: {len(vc_pr1)}")

# #Once a day metrics
# v_data_first_once_day = max(
#                    first_number_rh1,
#                    first_number_rh2,
#                    first_number_rh3)
# print(v_data_first_once_day)

# #24 times a day metrics
# v_data_first_24h_day = max(first_number_at1,
#                    first_number_pr1)
# print(v_data_first_24h_day)


#These vectors all start with a numeric value so missing values can be filled


#print(first_number_at1)
#print(first_number_rh1)
#print(first_number_rh2)
#print(first_number_rh3)
#print(first_number_pr1)



In [ ]:
df = pd.DataFrame(jj)
fc = df.iloc[:,0].astype('float32')
print(fc.shape)
import matplotlib.pyplot as plt

plt.hist(fc, bins=50, color='blue', alpha=0.7, density=True)
plt.title('Normalized Histogram of a')
plt.xlabel('Value')
plt.ylabel('Density')
plt.show()

In [ ]:
# Custom bins

a = fc[~fc.isnull()]

# Calculate min, max, and percentiles by 10 for 'a'
min_val = a.min()
max_val = a.max()
percentiles = np.percentile(a, np.arange(0, 101, 10))

print(f"Min: {min_val}")
print(f"Max: {max_val}")
print(f"Percentiles: {percentiles}")
